# Avaliação de Qualidade do Chatbot

In [ ]:
import os, time, json, asyncio
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from app.chatbot_agent import ChatbotAgent
load_dotenv()
agent = ChatbotAgent()


In [ ]:
questions = [
    "Olá, tudo bem?",
    "Qual é a capital do Brasil?",
    "Me ajuda com um problema?",
    "Como funciona isso?",
] + [f"Pergunta genérica {i}" for i in range(1, 47)]
len(questions)


In [ ]:
def ask(agent, q):
    class Input:
        def __init__(self, text):
            self.messages = [{"role": "user", "content": text}]
    return asyncio.run(agent.call(Input(q)))

rows = []
for q in questions:
    start = time.time()
    resp = ask(agent, q)
    elapsed_ms = (time.time()-start)*1000
    rows.append({"question": q, "response": resp.get("content", ""), "latency_ms": elapsed_ms})
df = pd.DataFrame(rows)
df.head()


In [ ]:
def is_pt(text):
    return any(w in text.lower() for w in ["você", "eu", "de", "para"])

def quality_checks(row):
    r = row["response"]
    respondeu = len(r.strip()) > 0
    pt_correto = is_pt(r)
    sem_erro = True
    tamanho_ok = 50 <= len(r) <= 2000
    nao_vazia = len(r.strip()) > 0
    checks = [respondeu, pt_correto, sem_erro, tamanho_ok, nao_vazia]
    return sum(checks), checks

scores = []
for _, row in df.iterrows():
    s, checks = quality_checks(row)
    scores.append(s)
df["quality_score"] = scores
df.head()


In [ ]:
score_final = (df["quality_score"].sum() / (len(df)*5)) * 100
taxa_sucesso = (df["quality_score"] >= 4).mean() * 100
latencia_media = df["latency_ms"].mean()
latencia_max = df["latency_ms"].max()
print("===== CHATBOT QUALITY REPORT =====")
print(f"Score Final: {score_final:.2f}/100")
print(f"Taxa de Sucesso: {taxa_sucesso:.2f}%")
print(f"Latência Média: {latencia_media:.2f} ms")
print(f"Latência Máxima: {latencia_max:.2f} ms")
print("================================")


In [ ]:
plt.figure(figsize=(10,4))
plt.hist(df["quality_score"], bins=6)
plt.title("Distribuição de quality_score")
plt.show()

plt.figure(figsize=(10,4))
plt.plot(df["latency_ms"])
plt.title("Latência por pergunta")
plt.show()

plt.figure(figsize=(10,4))
plt.scatter(df["quality_score"], df["latency_ms"])
plt.title("Taxa de sucesso vs Score (proxy)")
plt.show()


In [ ]:
from datetime import datetime
Path = __import__("pathlib").Path
ts = datetime.now().strftime("%Y-%m-%d_%H%M")
Path("results").mkdir(exist_ok=True)
out = {
  "score_final": float(score_final),
  "taxa_sucesso": float(taxa_sucesso),
  "latencia_media": float(latencia_media),
  "latencia_maxima": float(latencia_max),
  "respostas": df.to_dict(orient="records")
}
out_path = Path(f"results/chatbot_eval_{ts}.json")
out_path.write_text(json.dumps(out, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"Salvo em: {out_path}")
